In [1]:
import os
import logging
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.serving import Model

In [2]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [7]:
estimator = TensorFlow(entry_point='model.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             #framework_version='2.2.0',
                             py_version='py3',
                             source_dir='./train',
                             image_name='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.2.0-gpu-py37-cu102-ubuntu18.04',
                             distributions={'parameter_server': {'enabled': True}})

In [8]:
estimator.fit()

2020-07-19 18:02:58 Starting - Starting the training job...
2020-07-19 18:03:00 Starting - Launching requested ML instances......
2020-07-19 18:04:05 Starting - Preparing the instances for training...
2020-07-19 18:04:50 Downloading - Downloading input data
2020-07-19 18:04:50 Training - Downloading the training image.........
2020-07-19 18:06:22 Training - Training image download completed. Training in progress..2020-07-19 18:06:25,661 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-07-19 18:06:25,668 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-19 18:06:25,940 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/usr/local/bin/python3.7 -m pip install -r requirements.txt
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3754 sha256=553d64d8f903ef2c5abd45bbc26f9f34a4386d89d0aa11f4292cac79e84aed59
  Stored in directory: /root/.cache/pip/wheel

In [47]:
role

'arn:aws:iam::378706508412:role/service-role/AmazonSageMaker-ExecutionRole-20200607T125385'

In [9]:
model_location = 's3://sagemaker-us-west-2-378706508412/tensorflow-training-2020-07-19-18-02-57-858/output/model.tar.gz'

deployment_model = Model(
              entry_point='inference.py',
              model_data=model_location,
              #framework_version='2.2.0'
              source_dir='./serve',
              container_log_level = logging.DEBUG,
              image='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:2.2.0-gpu-py37-cu102-ubuntu18.04',
              #image='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:1.15.2-gpu-py36-cu100-ubuntu18.04',
              role=role)

predictor = deployment_model.deploy(initial_instance_count=1, instance_type='ml.t2.2xlarge')


-----------------!

In [10]:
predictor.predict({'claim_text': 'masks spread the coronavirus'})

{'predictions': [[2.53757366e-06]]}

In [11]:
predictor.endpoint

'tensorflow-inference-2020-07-19-18-11-28-675'